In [ ]:
!pip install git+https://github.com/cosmoquester/transformers-tf-finetune.git

In [ ]:
import csv
import random
import urllib.request

import tensorflow as tf
from transformers import AutoTokenizer, TFAutoModel

from transformers_tf_finetune.losses import PearsonCorrelationLoss
from transformers_tf_finetune.metrics import (
    BinaryF1Score,
    PearsonCorrelationMetric,
    SpearmanCorrelationMetric,
    pearson_correlation_coefficient,
    spearman_correlation_coefficient,
)
from transformers_tf_finetune.models import SemanticTextualSimailarityWrapper
from transformers_tf_finetune.utils import LRScheduler, get_device_strategy, path_join, set_random_seed

# Config

In [ ]:
#: transformers pretrained path
pretrained_model = "cosmoquester/bart-ko-small"
#: pretrained tokenizer fast pretrained path
pretrained_tokenizer = "cosmoquester/bart-ko-small"
#: load from pytorch weight
from_pytorch = False
#: use huggingface credential for private model
use_auth_token = ""

train_dataset_path = "https://raw.githubusercontent.com/kakaobrain/KorNLUDatasets/master/KorSTS/sts-train.tsv"
dev_dataset_path = "https://raw.githubusercontent.com/kakaobrain/KorNLUDatasets/master/KorSTS/sts-dev.tsv"
test_dataset_path = "https://raw.githubusercontent.com/kakaobrain/KorNLUDatasets/master/KorSTS/sts-test.tsv"
#: output directory to save log and model checkpoints, should be GCS path with TPU
output_path = None

#: training params
epochs = 5
learning_rate = 5e-5
min_learning_rate = 1e-5
warmup_rate = 0.06
warmup_steps = None
batch_size = 128
dev_batch_size = 512
tensorboard_update_freq = 1

#: device to use (TPU or GPU or CPU)
device = "TPU"
#: Use mixed precision FP16
mixed_precision = False
#: Set random seed
seed = None

In [ ]:
if output_path is not None and output_path.startswith("gs://"):
  from google.colab import auth
  auth.authenticate_user()

In [ ]:
def load_dataset(dataset_path: str, tokenizer: AutoTokenizer, shuffle: bool = False) -> tf.data.Dataset:
    """
    Load KorSTS dataset from local file or web

    :param dataset_path: local file path or file uri
    :param tokenizer: PreTrainedTokenizer for tokenizing
    :param shuffle: whether shuffling lines or not
    :returns: KorSTS dataset, number of dataset
    """
    if dataset_path.startswith("https://"):
        with urllib.request.urlopen(dataset_path) as response:
            data = response.read().decode("utf-8")
    else:
        with open(dataset_path) as f:
            data = f.read()
    lines = data.splitlines()[1:]
    if shuffle:
        random.shuffle(lines)

    start_token = tokenizer.bos_token or tokenizer.cls_token or ""
    end_token = tokenizer.eos_token or tokenizer.sep_token or ""

    sentences1 = []
    sentences2 = []
    normalized_labels = []
    for *_, score, sentence1, sentence2 in csv.reader(lines, delimiter="\t", quoting=csv.QUOTE_NONE):
        sentences1.append(start_token + sentence1 + end_token)
        sentences2.append(start_token + sentence2 + end_token)
        normalized_labels.append(float(score) / 5.0)

    inputs1 = dict(
        tokenizer(
            sentences1, padding=True, return_tensors="tf", return_token_type_ids=False, return_attention_mask=True,
        )
    )
    inputs2 = dict(
        tokenizer(
            sentences2, padding=True, return_tensors="tf", return_token_type_ids=False, return_attention_mask=True,
        )
    )

    dataset = tf.data.Dataset.from_tensor_slices(((inputs1, inputs2), normalized_labels))
    return dataset

In [ ]:
if seed:
    set_random_seed(seed)

In [ ]:
strategy = get_device_strategy(device)

# Mixed Precision

In [ ]:
with strategy.scope():
    if mixed_precision:
        mixed_type = "mixed_bfloat16" if device == "TPU" else "mixed_float16"
        policy = tf.keras.mixed_precision.experimental.Policy(mixed_type)
        tf.keras.mixed_precision.experimental.set_policy(policy)

# Load Dataset

In [ ]:
with strategy.scope():
    tokenizer = AutoTokenizer.from_pretrained(pretrained_tokenizer, use_auth_token=use_auth_token)

    train_dataset = load_dataset(train_dataset_path, tokenizer, True)
    train_dataset = train_dataset.batch(batch_size)
    dev_dataset = load_dataset(dev_dataset_path, tokenizer).batch(dev_batch_size)
    test_dataset = load_dataset(test_dataset_path, tokenizer).batch(dev_batch_size)

# Load Model

In [ ]:
with strategy.scope():
    model = TFAutoModel.from_pretrained(
        pretrained_model, use_auth_token=use_auth_token, from_pt=from_pytorch
    )
    model_sts = SemanticTextualSimailarityWrapper(model=model, embedding_dropout=0.1)

# Model Compile

In [ ]:
with strategy.scope():
    model_sts.compile(
        optimizer=tf.keras.optimizers.Adam(
            LRScheduler(
                len(train_dataset) * epochs,
                learning_rate,
                min_learning_rate,
                warmup_rate,
                warmup_steps,
            ),
        ),
        loss=[PearsonCorrelationLoss(), tf.keras.losses.MeanSquaredError()],
        loss_weights=[0.25, 0.75],
        metrics=[
            BinaryF1Score(),
            PearsonCorrelationMetric(name="pearson_coef"),
            SpearmanCorrelationMetric(name="spearman_coef"),
        ],
    )

# Model Training

In [ ]:
with strategy.scope():
    model_sts.fit(
        train_dataset,
        validation_data=dev_dataset,
        epochs=epochs,
        callbacks=[
            tf.keras.callbacks.ModelCheckpoint(
                path_join(output_path, "best_model.ckpt"),
                save_weights_only=True,
                save_best_only=True,
                monitor="val_spearman_coef",
                mode="max",
                verbose=1,
            ),
            tf.keras.callbacks.TensorBoard(
                path_join(output_path, "logs"), update_freq=tensorboard_update_freq
            ),
        ] if output_path is not None else None,
    )

# Model Evaluate

In [ ]:
with strategy.scope():
    preds = []
    labels = []
    f1 = BinaryF1Score()
    for inputs, label in test_dataset:
        pred = model_sts(inputs, training=False)
        preds.extend(pred.numpy())
        labels.extend(label.numpy())
        f1.update_state(label, pred)

    pearson_score = pearson_correlation_coefficient(labels, preds)
    spearman_score = spearman_correlation_coefficient(labels, preds)
    print(
        f"Dev F1 Score: {f1.result():.4f}, "
        f"Dev Pearson: {pearson_score:.4f}, "
        f"Dev Spearman: {spearman_score:.4f}"
    )